# Assessment Problems
---
This notebook explores the key components of the SHA-256 cryptographic hash algorithm as defined in the [Secure Hash Standard (FIPS 180-4)](https://nvlpubs.nist.gov/nistpubs/fips/nist.fips.180-4.pdf). It systematically builds SHA-256 from its fundamental bitwise operations through the complete compression function.

## Table of Contents
* Problem 1: Binary Words and Operations
* Problem 2: Fractional Parts of Cube Roots
* Problem 3: Padding
* Problem 4: Hashes
* Problem 5: Passwords
    
All implementations follow FIPS 180-4 specifications and include comprehensive testing with verification against official standards."

## Imports

In [29]:
# NumPy.
import numpy as np
from typing import Generator 

# Haslib for Problem 4 & Problem 5
import hashlib 

## Problem 1: Binary Words and Operations

### Technical Glossary
This section defines operations & technical terms used throughout the implementation of solutions for the problems set out in the assignment.

### Bitwise Logical Operations

**Bitwise Operations** are operations that work on the individiual bits of a binary number. Applying the operator to each bit individually but all bits simulatenously.

**XOR (Exclusive OR)**
Outputs 1 when the input bits are different.

| A | B | A ⊕ B|
|---|---|-------|
| 0 | 0 |   0   |
| 0 | 1 |   1   |
| 1 | 0 |   1   |
| 1 | 1 |   0   |

**AND**
Outputs 1 when both input bits are 1

| A | B | A & B|
|---|---|-------|
| 0 | 0 |   0   |
| 0 | 1 |   0   |
| 1 | 0 |   0   |
| 1 | 1 |   1   |

**NOT**
Output flips the bits

| A | ¬A|
|---|---|
|0  |1  |
|1  |0  |

**Bit Manipulation Operations**
* `ROTR^n(x)`: Rotate right - shifts bits to the right n times, 'fallen' bits move back to the left (start)
* `SHR^n(x)`: Logical shift right - shifts bits to the right n times; fills left with zeros

**Common Terms**
* **Bitwise Operation:** Performs an operation on each individual bit
* **32-bit unsigned integer:** An integer represented by 32 individual bits in a binary sequence. "Unsigned" meaning non-negative values.
* **Diffusion:** Where the input changes are spread throughout the output

See: [FIPS 180-4, Sections 2.2.2 and 3.2](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

### Implementing Parity
**Spec:** Defined in the Secure Hash Standard [Page 10](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

> $\text{Parity}(x, y, z) = x \oplus y \oplus z$

**How it works:**
Chains $'\oplus'$ operations across three 32-bit integers.

Since $'\oplus'$ is a binary operation it takes 2 inputs. The expression is evaluated as $ \text (x \oplus y \oplus z)$
. The result of $x \oplus y$ is then, result $\oplus z$.

Each output bit is 1 if there is an odd number of input bits set to 1, otherwise 0. This is why its called "Parity" it checks the parity(odd/even count) of bits at each position.

**Example**

X = 1100

Y = 1100

Z = 0110

Output = 0110

In [30]:
def Parity(x, y, z) -> np.uint32:
    """
    Calculate the parity (bitwise XOR) of three 32-bit unsigned integers.

    Args:
        x, y, z (int): 32-bit unsigned integer.

    Returns:
        int: The parity of the three integers as a 32-bit unsigned integer.

    """
    
    # Loop over each passed parameters
    for args in (x, y, z):
        # Use isInstance to ensure correct type (object, desired type)
        # See: https://www.w3schools.com/python/ref_func_isinstance.asp
        if not isinstance(args, (int, np.integer)):
            # Raises / Throws a TypeError with a message
            raise TypeError("All arguments must be integers. No other types are allowed.")

    # Cast all inputs as NumPy unsigned 32-bit integers.
    # See: https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)

    # Apply bitwise XOR (^) to the three inputs and return
    return x ^ y ^ z

In [31]:
def testing_Parity():
    assert Parity(1, 0, 0) == np.uint32(1)
    assert Parity(1, 1, 0) == np.uint32(0)
    assert Parity(7, 13, 5) == np.uint32(15)

def test_error_on_Parity():
    try:
        Parity(1.5, 2, 3)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")
    
testing_Parity()
test_error_on_Parity()

print("All test cases pass on Parity")

All test cases pass on Parity


### Implementing Choose (Ch)
**Spec:** Defined in the Secure Hash Standard [Page 10](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

> $\text{Ch}(x, y, z) = (x \land y) \oplus (\lnot x  \land  z)$

__How it works:__
The $\text{Ch}(x, y, z)$ uses x as a selector mask. For each bit position, if x is 1, the output takes the bit from y.
If x is 0, the output takes the bit from z.

This is achieved by calculating $\text (x \land y)$ to select bits from y where x is 1, then $\text (\lnot x \land y)$ to select bits from z where x is 0. Then $'\oplus'$ the results together.

**Example**

X = 1110 

Y = 1010 

Z = 1111 

Output = 1011

In [32]:
def Ch(x: int, y: int, z: int) -> np.uint32:
    """
    Bitwise Choose: return bits from y where x is 1, and bits from z where x is 0.

    Parameters
    ----------
        x, y, z : int
            32-bit unsigned integers.

    Returns
    -------
        numpy.uint32
            The output 'masked' value of the three integers.

    Raises
    ------
        TypeError
            If any argument is not an integer.
    """

    # Loop over the parameters passed
    for args in (x, y, z):
        # Use isinstance + raise TypeError as before
        # See: Problem 1 -> Implementing Parity
        if not isinstance(args, (int, np.integer)):
            raise TypeError("All arguments must be integers. No other types are allowed.")

    # Cast values as before, see: Problem 1 -> Implementing Parity
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)

    # Perform the bitwise operation and return the value
    return (x & y) ^ (~x & z)

In [33]:
def testing_Ch():
    assert Ch(1, 0, 0) == np.uint32(0)
    assert Ch(1, 3, 6) == np.uint32(7)
    assert Ch(3, 5, 8) == np.uint32(9)

def test_error_on_Ch():
    try:
        Ch(2.5, 4, 9)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")
    
testing_Ch()
test_error_on_Ch()

print("All test cases pass on Ch")

All test cases pass on Ch


### Implementing Maj
**Purpose:** The Maj computes the bitwise (XOR, AND) of 3 unsigned 32-bit integers. 

**Spec:** Defined in the Secure Hash Standard [Page 10](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

> $\text{Maj}(x, y, z) = (x \land y) \oplus (x \land  z) \oplus (y \land  z)$

**How it works:**
The $\text{Maj}(x, y, z)$ implements majority voting at each bit position. For each bit, the output is 1 if at least two input bits are 1, otherwise 0.
This is computed by taking the $'\land'$ of each pair of inputs 
$\text (x \land y)$, $\text (x \land z)$, and $\text (y \land z)$, then $'\oplus'$ the three results together.

**Example:**

X = 11101110

Y = 10100101

Z = 11111001

Output = 11101101

In [34]:
def Maj(x: int, y: int, z: int) -> np.uint32:
    """
    Bitwise Majority: output bit is 1 if two or more input bits are 1.

    Parameters
    ----------
        x, y, z : int
            32-bit unsigned integers.

    Returns
    -------
        numpy.uint32
            The output 'voted' values of the three integers.

    Raises
    ------
        TypeError
            If any argument is not an integer.
    """

    # Loop over args, use isinstance & raise TypeError as before
    # See: Problem 1 -> Implementing Parity
    for args in (x, y, z):
        if not isinstance(args, (int, np.integer)):
            raise TypeError("All arguments must be integers. No other types are allowed.")
    
    # Cast values as before, see: Problem 1 -> Implementing Parity
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)

    # Perform the bitwise operation & return the value
    return (x & y) ^ (x & z) ^ (y & z)

In [35]:
def testing_Maj():
    assert Maj(1, 2, 3) == np.uint32(3)
    assert Maj(3, 4, 7) == np.uint32(7)
    assert Maj(5, 8, 10) == np.uint32(8)

def test_error_on_Maj():
    try:
        Maj(2, 4.7, 6)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")
    
testing_Maj()
test_error_on_Maj()

print("All test cases pass for Maj")

All test cases pass for Maj


### Implementing Big & Small Sigmas with Helper Functions
#### Helper Functions

__Purpose:__
Support functions used in both big and small sigmas.

* `ROTR(x, n)`: Rotate right - shifts bits to the right n times, 'fallen' bits move back to the left (start)
* `SHR(x, n)`: Logical shift right - shifts bits to the right n times; fills left with zeros

__Spec.__ Defined in the Secure Hash Standard (FIPS 180-4, p.8-9) as:

$\text {ROTR}^n(x) = (x \gg n) \lor (x \ll (w - n))$

$\text {SHR}^n(x) = x \gg n$

See: [FIPS 180-4, Sections 2.2.2](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

__Intuition:__
Both functions provide structured bit-mixing.

* Rotate shifts the bits without losing information.
* Logical shift loses information but adds to diffusion.

__Example__

x = 10110011

ROTR(x, 2) = 11101100

SHR(x, 4) = 00001011

In [36]:
def ROTR(x: int, n: int) -> np.uint32:
    """
    Perform a right rotation on a 32-bit unsigned integer.

    Parameters
    ----------
        x : int
            32-bit unsigned integer to be rotated.
        n : int
            Number of positions to rotate.

    Returns
    -------
        numpy.uint32
            The result of the right rotation as a 32-bit unsigned integer.
    
    Raises
    ------
        TypeError
            If any argument is not an integer.
    """

    # Type check as before
    for args in (x, n):
        if not isinstance(args, (int, np.integer)):
            raise TypeError("All arguments must be integers. No other types are allowed.")
    
    # Early return if n is 0
    if n == 0:
        return x
    
    # Ensure n is within the range of 0-31
    n = n % 32

    # Cast value to ensure correct type and prevent overflow errors
    x = np.uint32(x)

    # Perform the rotation and return the value
    return (x >> n) | (x << (32 - n))

In [37]:
def SHR(x: int, n: int) -> np.uint32:
    """
    Perform a logical right shift on a 32-bit unsigned integer.

    Parameters
    ----------
        x : int 
            32-bit unsigned integer to be shifted.
        n : int
            Number of positions to shift.

    Returns
    -------
        numpy.uint32
            The result of the logical right shift as a 32-bit unsigned integer.
    """

    # Type check as before
    for args in (x, n):
        if not isinstance(args, (int, np.integer)):
            raise TypeError("All arguments must be integers. No other types are allowed.")
        
    # Early return if n is 0
    if n == 0:
        return x
    
    # Ensure n is within the range of 0-31
    n = n % 32

    # Cast value to ensure correct type and prevent overflow errors
    x = np.uint32(x)

    # Perform the logical right shift and return the value
    return x >> n

In [38]:
# Same format for tests as previous sub-problems
def test_ROTR_valid():
    assert ROTR(10, 1) == np.uint32(5)
    assert ROTR(15, 2) == np.uint32(3221225475)
    assert ROTR(18, 3) == np.uint32(1073741826)

def test_SHR_valid():
    assert SHR(10, 1) == np.uint32(5)
    assert SHR(13, 2) == np.uint32(3)

def test_ROTR_type_error():
    try:
        ROTR(1.5, 2)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")

def test_SHR_type_error():
    try:
        SHR(1.5, 2)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")

test_ROTR_valid()
test_SHR_valid()
test_ROTR_type_error()
test_SHR_type_error()

print("All test cases pass on ROTR & SHR")

All test cases pass on ROTR & SHR


#### Big Sigma 0 - $\Sigma_0^{\{256\}}(x)$

**Purpose**:
Provides non-linear mixing by combining three rotated versions of the input. This ensures the bits are widely spread across. Increasing diffusion.
    
**Spec.** Defined in the Secure Hash Standard as:
    
> $\Sigma_0^{\{256\}}(x) = \text{ROTR}^{2}(x) \oplus \text{ROTR}^{13}(x) \oplus \text{ROTR}^{22}(x)$

See: [FIPS 180-4, Sections 4.1.2](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

**Intuition:**

Rotating the words by different amounts makes sure each bit affect multiple output bit positions.
XOR-ing the results ensures the output is very complex to reverse.

**Example**

x = 10110011

Each bit is rotated **2**, **13** & **22** times, then XOR'ed together.

These numbers are **not** arbitrary they are a result of testing during the creation of the Secure Hash Standard.

They reinforce the avalanche effect & ensure good spread diffusion and mixing.

In [39]:
def bigSigma0(x: int) -> np.uint32:
    """
    Performs the Σ₀ function on a 32-bit word.

    This function is used in:
    - The SHA-256 compression function 
    - The SHA-224 compression function

    The operation involves three circular right rotations (ROTR) of the input 
    word by 2, 13, and 22 bits, followed by a bitwise XOR of the results.

    Parameters:
        x (int): A 32-bit word.

    Returns:
        np.uint32: The result of ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x), as a 32-bit unsigned integer.
    """
    
    # Type check as before
    if not (isinstance(x, (int, np.uint32))):
        raise TypeError("Input must be a non-negative integer")
    
    # Cast value to ensure correct type and prevent overflow errors
    x = np.uint32(x)

    # Perform the Big Sigma 0 operation and return the value
    return np.uint32(ROTR(x, 2) ^ ROTR(x, 13) ^ ROTR(x, 22))

#### Big Sigma 1 - $\Sigma_1^{\{256\}}(x)$

**Purpose**:
Exact same function as Sigma 0, except with different rotation constants, providing another layer of complexity, diffusion and mixing.
    
**Spec.** Defined in the Secure Hash Standard as:
    
> $\Sigma_1^{\{256\}}(x) = \text{ROTR}^{6}(x) \oplus \text{ROTR}^{11}(x) \oplus \text{ROTR}^{25}(x)$

See: [FIPS 180-4, Sections 4.1.2](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

**Intuition:**

Rotating the words by different amounts makes sure each bit affect multiple output bit positions.
XOR-ing the results ensures the output is very complex to reverse.

**Example**

x = 10110011

Each bit is rotated **6**, **11** & **25** times, then XOR'ed together.

These numbers are **not** arbitrary they are a result of testing during the creation of the Secure Hash Standard.

They reinforce the avalanche effect & ensure good spread diffusion and mixing.

In [40]:
def bigSigma1(x: int) -> np.uint32:
    """
    Performs the Σ₁ function on a 32-bit word.

    This function is used in:
    - The SHA-256 compression function
    - The SHA-224 compression function

    The operation involves three circular right rotations (ROTR) of the input
    word by 6, 11, and 25 bits, followed by a bitwise XOR of the results.

    Parameters:
        x (int): A 32-bit word.

    Returns:
        np.uint32: The result of ROTR⁶(x) ⊕ ROTR¹¹(x) ⊕ ROTR²⁵(x), as a 32-bit unsigned integer.
    """
    
    # Type check as before
    if not isinstance(x, (int, np.integer)):
        raise TypeError("Argument must be an integer. No other types are allowed.")

    # Cast to np.uint32 to ensure correct type as before
    x = np.uint32(x)

    # Perform the Big Sigma 1 operation and return the value
    return np.uint32(ROTR(x, 6) ^ ROTR(x, 11) ^ ROTR(x, 25))

##### Test Cases (Big Sigma(s))

Checking:

- Correct mathematical implementation by comparing with manual calculations using helper functions
- Type enforcement (raise TypeError for non-integer values)
- Validation for both Big Sigma 0 and Big Sigma 1 functions

In [41]:
def test_bigSigma_0_valid():
    assert bigSigma0(1)  == np.uint32(ROTR(1, 2) ^ ROTR(1, 13) ^ ROTR(1, 22))
    assert bigSigma0(10) == np.uint32(ROTR(10, 2) ^ ROTR(10, 13) ^ ROTR(10, 22))

def test_bigSigma_1_valid():
    assert bigSigma1(7)  == np.uint32(ROTR(7, 6) ^ ROTR(7, 11) ^ ROTR(7, 25))
    assert bigSigma1(18) == np.uint32(ROTR(18, 6) ^ ROTR(18, 11) ^ ROTR(18, 25))

def test_bigSigma_type_error():
    try:
        bigSigma0(1.5)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")
    
    try:
        bigSigma1(1.5)
    except TypeError:
        pass
    else:
         raise AssertionError("Expected TypeError for float input")

test_bigSigma_0_valid()
test_bigSigma_1_valid()
test_bigSigma_type_error()

print("All test cases pass on Big Sigma 0 & 1")

All test cases pass on Big Sigma 0 & 1


#### Small Sigma 0 - $\sigma_0^{\{256\}}(x)$
* __Small Sigma 0__  -> $'\oplus'$ Bitwise, 2 rotations (7, 18) & Bitwise shift right (3) - SHA-256 Reference required here

**Spec.** Defined in the Secure Hash Standard as:
> $\sigma_0^{\{256\}}(x) = \text{ROTR}^{7}(x) \oplus \text{ROTR}^{18}(x) \oplus \text{SHR}^{3}(x)$

See: [FIPS 180-4, Sections 4.1.2](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

In [42]:
def smallSigma0(x: int) -> np.uint32:
    """
    Performs the σ₀ function on a 32-bit word.

    This function is used in the SHA-256 and SHA-224 message schedule expansion.

    The function performs two circular right rotations (ROTR) of the input
    by 7 and 18 positions, one right shift (SHR) by 3 positions, followed 
    by a bitwise XOR of the results.
    
    Parameters:
        x (int): A 32-bit word
        
    Returns:
        np.uint32: The result of ROTR⁷(x) ⊕ ROTR¹⁸(x) ⊕ SHR³(x), as a 32-bit unsigned integer
    """
    
    # Type check as before
    if not isinstance(x, (int, np.integer)):
        raise TypeError("Argument must be an integer. No other types are allowed.")
    
    # Cast to np.uint32 to ensure correct type as before
    x = np.uint32(x)

    # Perform the Small Sigma 0 operation and return the value
    return np.uint32(ROTR(x, 7) ^ ROTR(x, 18) ^ SHR(x, 3))

#### Small Signma 1 - $\sigma_1^{\{256\}}(x)$
* __Small Sigma 1__  -> $'\oplus'$ Bitwise, 2 rotations (17, 19) & Bitwise shift right (10) - SHA-256 Reference required here

**Spec.** Defined in the Secure Hash Standard as:
> $\sigma_1^{\{256\}}(x) = \text{ROTR}^{17}(x) \oplus \text{ROTR}^{19}(x) \oplus \text{SHR}^{10}(x)$

See: [FIPS 180-4, Sections 4.1.2](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

In [43]:
def smallSigma1(x: int) -> np.uint32:
    """
    Performs the σ₁ function on a 32-bit word.

    This function is used in the SHA-256 and SHA-224 message schedule expansion.

    The function performs two circular right rotations (ROTR) of the input
    by 17 and 19 positions, one right shift (SHR) by 10 positions, followed 
    by a bitwise XOR of the results.
    
    Parameters:
        x (int): A 32-bit word
        
    Returns:
        np.uint32: The result of ROTR¹⁷(x) ⊕ ROTR¹⁹(x) ⊕ SHR¹⁰(x), as a 32-bit unsigned integer
    """

    # Type check as before
    if not isinstance(x, (int, np.integer)):
        raise TypeError("Argument must be an integer. No other types are allowed.")

    # Cast to np.uint32 to ensure correct type as before
    x = np.uint32(x)

    # Perform the Small Sigma 1 operation and return the value
    return np.uint32(ROTR(x, 17) ^ ROTR(x, 19) ^ SHR(x, 10))

##### Test Cases (Small Sigma(s))

Checking:

- Correct mathematical implementation by comparing with manual calculations using helper functions
- Type enforcement (raise TypeError for non-integer values)
- Validation for both Small Sigma 0 and Small Sigma 1 functions

In [44]:
def test_smallSigma_0_valid():
    assert smallSigma0(1)  == np.uint32(ROTR(1, 7) ^ ROTR(1, 18) ^ SHR(1, 3))
    assert smallSigma0(10) == np.uint32(ROTR(10, 7) ^ ROTR(10, 18) ^ SHR(10, 3))

def test_smallSigma_1_valid():
    assert smallSigma1(5)  == np.uint32(ROTR(5, 17) ^ ROTR(5, 19) ^ SHR(5, 10))
    assert smallSigma1(12)  == np.uint32(ROTR(12, 17) ^ ROTR(12, 19) ^ SHR(12, 10))

# Error on this function like using floating numbers
def test_smallSigma_type_error():
    try:
        smallSigma0(1.5)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")
    
    try:
        smallSigma1(1.5)
    except TypeError:
        pass
    else:
         raise AssertionError("Expected TypeError for float input")

test_smallSigma_0_valid()
test_smallSigma_1_valid()
test_smallSigma_type_error()

print("All test cases pass Small Sigma 0 & 1")

All test cases pass Small Sigma 0 & 1


---

## Problem 2: Fractional Parts of Cube Roots

### Primes Function
**Purpose:** Every integer greater than 1 can be represented uniquely as a product of prime numbers. This could generates the 1st 64 prime numbers. Start with the 1st prime number and all the way to the 64th prime number. 

**Implementing Prime**
* Implementing a function to calculate the first n prime numbers

**How it works:**

- Performs input & type validation
- Create a list with first prime (2)
- Loops until the list of ``primes`` is length of ``n``
- Checks if the number is divisible by any of the current primes found
- Jumps 2 numbers to check at a time (all even numbers > 2 aren't prime)

References:

See: [GeeksForGeeks Implementation Of Calculating Finding N Prime Numbers](https://www.geeksforgeeks.org/dsa/generate-and-print-first-n-prime-numbers/)

**Example:** 
* 2, 3, 5 is a prime number.
* 4, 10, 12 is NOT a prime number because it's a factor number. Like 10 / 2 = 5
* 0, 1 is a Non-Prime number

In [45]:
def primes(n: int) -> list[int]:
    """
    Generate the first n prime numbers.

    Parameters
    ----------
        n : int

    Returns
    -------
        list[int]
            A list of the first n prime numbers.

    Raises
    ------
        TypeError
            If the argument is not an integer.
        ValueError
            If the argument is not a positive integer.
    """
    # Type check as before
    if not isinstance(n, (int, np.integer)):
        raise TypeError("Argument must be an integer.")
    
    # Ensure n is a positive integer
    if n <= 0:
        raise ValueError("Argument must be a positive integer.")

    # Start with the first prime number in a list
    primes = [2]

    # Start checking for primes from 3 onwards
    num = 3

    # Loop until we have calculated n primes
    while len(primes) < n:
        # Assume number is prime to start
        is_prime = True

        for i in range(len(primes)):
            # Check to see if current number is divisible by any previous primes
            if num % primes[i] == 0:
                # If it is, its not a prime number, skip to next number
                is_prime = False
                break
        
        # If number is still prime, add to list
        if is_prime:
            primes.append(num)

        # Skip even numbers, even numbers above 2 are not prime
        num += 2  

    # Return completed list
    return primes

#### Testing Prime

- Tests to see if the function can return the first N Prime numbers.
- Ensure the correct errors are being raised for passed args.

In [46]:
def test_prime_valid():
    assert primes(5) == [2, 3, 5, 7, 11]
    assert primes(10) == [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]

def test_prime_type_value_error():
    try:
        primes(1.5)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")
    
    try:
        primes(-5)
    except ValueError:
        pass
    else:
         raise AssertionError("Expected ValueError for negative input")
    
test_prime_valid()
test_prime_type_value_error()

print("All test cases pass")

All test cases pass


### Calculating the Cube Root
**Purpose:** After getting the first 64 prime numbers, collect the first 32 bits of fractional by calculating the cube root

* Code for calculating cube root of first 64 prime numbers per the brief

[For Calculating Cube Roots in Python](https://www.geeksforgeeks.org/python/python-program-for-find-cubic-root-of-a-number/)

**Example:**
> $\text{root} = {prime}^{1/3}$

> $\text{frac} = {root} - {int(root)}$

> $\text{frac} = {frac}({2}^{32})$

In [47]:
def calc_cube_roots(n: int) -> list[str]:
    """
    Calculate the cube roots of a list of prime numbers.

    Parameters
    ----------
        n : int
            The number of cube roots to calculate.

    Returns
    -------
        list[str]
            A list of hexadecimal strings representing the cube roots.

    Raises
    ------
        TypeError
            If the argument is not a list of integers.
    """
    # Type check as before
    if not isinstance(n, (int, np.integer)):
        raise TypeError("Argument must be an integer.")

    # Get the first n primes from function above
    primes_list = primes(n)

    # Create an empty list to store results
    hex_cube_roots = []

    # Loop over each prime number
    for p in primes_list:
        # Calculate the cube root using numpy.cbrt
        # See: https://numpy.org/doc/stable/reference/generated/numpy.cbrt.html
        cube_root = np.cbrt(p)

        # Extract the fractional part of the cube root
        fractional_part = cube_root % 1

        # Convert the fractional number to 32-bit by multiplying by 2^32
        full_32bit = int(fractional_part * (2**32))

        # Convert to hexadecimal and format to 8 characters
        # See: https://docs.python.org/3/library/functions.html#hex
        hex_value = format(full_32bit, '08x')

        # Append to results list
        hex_cube_roots.append(hex_value)

        # Print the results
        print(hex_value)

    # Return the list of strings
    return hex_cube_roots

#### Test Cases (Cube Roots)
**Testing:**
* Correct calculation of cube roots for first 64 primes
* Proper hexadecimal formatting (8 characters each)
* Type enforcement for invalid inputs
* Validation against SHA-256 K constants


See: [FIPS 180-4, Section 4.2.2, p.11](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

In [48]:
def test_cube_roots():
    # SHA-256 round constants (first 64 cube roots of primes)
    # See: FIPS 180-4, Section 4.2.2, p.11
    # https://nvlpubs.nist.gov/nistpubs/fips/nist.fips.180-4.pdf
    secure_hash_constants = ["428a2f98", "71374491", "b5c0fbcf", "e9b5dba5", "3956c25b", "59f111f1", "923f82a4", "ab1c5ed5",
        "d807aa98", "12835b01", "243185be", "550c7dc3", "72be5d74", "80deb1fe", "9bdc06a7", "c19bf174",
        "e49b69c1", "efbe4786", "0fc19dc6", "240ca1cc", "2de92c6f", "4a7484aa", "5cb0a9dc", "76f988da",
        "983e5152", "a831c66d", "b00327c8", "bf597fc7", "c6e00bf3", "d5a79147", "06ca6351", "14292967",
        "27b70a85", "2e1b2138", "4d2c6dfc", "53380d13", "650a7354", "766a0abb", "81c2c92e", "92722c85",
        "a2bfe8a1", "a81a664b", "c24b8b70", "c76c51a3", "d192e819", "d6990624", "f40e3585", "106aa070",
        "19a4c116", "1e376c08", "2748774c", "34b0bcb5", "391c0cb3", "4ed8aa4a", "5b9cca4f", "682e6ff3",
        "748f82ee", "78a5636f", "84c87814", "8cc70208", "90befffa", "a4506ceb", "bef9a3f7", "c67178f2"
    ]

    calculated_constants = calc_cube_roots(64)
    assert calculated_constants == secure_hash_constants, "Calculated constants do not match expected values."

def test_cube_roots_type_error():
    try:
        calc_cube_roots(1.5)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")

test_cube_roots()
test_cube_roots_type_error()
print("All test cases pass")

428a2f98
71374491
b5c0fbcf
e9b5dba5
3956c25b
59f111f1
923f82a4
ab1c5ed5
d807aa98
12835b01
243185be
550c7dc3
72be5d74
80deb1fe
9bdc06a7
c19bf174
e49b69c1
efbe4786
0fc19dc6
240ca1cc
2de92c6f
4a7484aa
5cb0a9dc
76f988da
983e5152
a831c66d
b00327c8
bf597fc7
c6e00bf3
d5a79147
06ca6351
14292967
27b70a85
2e1b2138
4d2c6dfc
53380d13
650a7354
766a0abb
81c2c92e
92722c85
a2bfe8a1
a81a664b
c24b8b70
c76c51a3
d192e819
d6990624
f40e3585
106aa070
19a4c116
1e376c08
2748774c
34b0bcb5
391c0cb3
4ed8aa4a
5b9cca4f
682e6ff3
748f82ee
78a5636f
84c87814
8cc70208
90befffa
a4506ceb
bef9a3f7
c67178f2
All test cases pass


---

## Problem 3: Padding

### Solution:
#### Implementing Block Parse
**Purpose:** SHA-256 processes messages in fixed-size blocks of 512 bits (64 bytes). Messages of arbitrary length must be padded to ensure the final length is a multiple of 512 bits. This function implements the padding scheme and yields message blocks one at a time.

**What is padding?** Padding adds extra data to a message to reach a required length. For SHA-256, padding ensures:

1. The message ends on a block boundary (multiple of 512 bits)
2. The original message length is preserved and authenticated
3. Different messages produce different padded results (no collisions from padding alone)

**Specification** (FIPS 180-4, Sections 5.1.1 and 5.2.1):

The padding process:

1. Append a single '1' bit (byte 0x80 = 10000000 in binary)
2. Append '0' bits until message length ≡ 448 mod 512 (leaves 64 bits remaining)
3. Append original message length as 64-bit big-endian integer
4. Split padded message into 512-bit blocks

**Why this padding scheme?**

* The '1' bit ensures there's always at least some padding (prevents ambiguity)
* The length field prevents length-extension attacks
* The specific modulo ensures exactly 64 bits remain for the length

**Example:**

Original message: "abc" (24 bits)

_Step 1_ - Append '1' bit: "abc" + 0x80

_Step 2_ - Append zeros: Pad to 448 bits (56 bytes)

_Step 3_ - Append length: Add 0x0000000000000018 (24 in 64-bit)

_Result:_ Single 512-bit block

**Generator function:** This implementation uses a Python generator, which yields blocks one at a time rather than returning all blocks at once. This is memory-efficient for large messages.

**Jargon explanation:**

* **Generator:** A function that uses yield to produce values one at a time, pausing between each
* **Big-endian:** Most significant byte first (standard network byte order)
* **mod 512:** Remainder when divided by 512

In [49]:
def block_parse(msg):
    """
    Generator function that parses a message into 512-bit blocks with SHA-256 padding.
    
    Implements padding according to sections 5.1.1 and 5.2.1 of the Secure Hash Standard.
    
    Parameters:
        msg (bytes): The input message to be parsed
        
    Yields:
        bytes: Each 512-bit (64-byte) block of the padded message
    """
    # Length of original message in bits
    msg_len_bits = len(msg) * 8
    
    # Convert message to mutable bytearray for padding 
    padded_msg = bytearray(msg)
    
    # Append the '1' bit (10000000 in binary, or 0x80 in hex)
    padded_msg.append(0x80)
    
    # Calculate the length mod 64 to determine padding needed to reach 56 bytes (448 bits)
    padding_mod = len(padded_msg) % 64
    padding_needed = (56 - padding_mod) if padding_mod <= 56 else (120 - padding_mod)
    
    # Append zero bytes to reach the required length before length encoding
    padded_msg.extend([0x00] * padding_needed)
    
    # Append the original message length as a 64-bit big-endian integer
    padded_msg.extend(msg_len_bits.to_bytes(8, byteorder='big'))
    
    # Yield successive 512-bit (64-byte) blocks
    for i in range(0, len(padded_msg), 64):
        yield bytes(padded_msg[i:i+64])

##### Test Cases (Padding)
* Correct padding for messages of various lengths (short messages, messages near block boundaries)
* Proper block size validation (all output blocks are exactly 64 bytes/512 bits)
* Type enforcement for non-bytes input and verification against known test vectors

In [50]:
def test_block_parse():
    # Test 1: Empty message
    msg1 = b"" 
    blocks1 = list(block_parse(msg1)) # Convert generator to list for testing
    assert len(blocks1) == 1, "Empty message should produce 1 block"
    expected_block1 = b'\x80' + b'\x00' * 55 + (0).to_bytes(8, 'big') # \x80 is 10000000 in binary, \x00*55 pads to 56 bytes, last 8 bytes are length (0 bits)
    assert blocks1[0] == expected_block1, "Padding for empty message incorrect"

    # Test 2: Short message "abc"
    msg2 = b"abc"
    blocks2 = list(block_parse(msg2))
    assert len(blocks2) == 1, "3-byte message should produce 1 block"
    padding_len = 56 - (len(msg2) + 1)  # +1 for 0x80 byte
    expected_block2 = msg2 + b'\x80' + b'\x00' * padding_len + (len(msg2)*8).to_bytes(8, 'big')
    assert blocks2[0] == expected_block2, "Padding for short message incorrect"

    # Test 3: Boundary case - 55-byte message (one block)
    msg3 = b'a' * 55
    blocks3 = list(block_parse(msg3))
    assert len(blocks3) == 1, "55-byte message should produce 1 block"
    padding_len = 56 - (len(msg3) + 1)
    expected_block3 = msg3 + b'\x80' + b'\x00' * padding_len + (len(msg3)*8).to_bytes(8, 'big')
    assert blocks3[0] == expected_block3, "Padding for 55-byte message incorrect"

    # Test 4: Boundary case - 56-byte message (requires 2 blocks)
    msg4 = b'a' * 56
    blocks4 = list(block_parse(msg4))
    assert len(blocks4) == 2, "56-byte message should produce 2 blocks"

    expected_block4_0 = msg4 + b'\x80' + b'\x00' * 7
    expected_block4_1 = b'\x00' * 56 + (len(msg4)*8).to_bytes(8, 'big')

    assert blocks4[0] == expected_block4_0, "First block for 56-byte message incorrect"
    assert blocks4[1] == expected_block4_1, "Padding block for 56-byte message incorrect"

    # Test 5: Exact 64-byte message (2 blocks)
    msg5 = b'b' * 64
    blocks5 = list(block_parse(msg5))
    assert len(blocks5) == 2, "64-byte message should produce 2 blocks"

    expected_block5_0 = msg5
    expected_block5_1 = b'\x80' + b'\x00' * 55 + (len(msg5)*8).to_bytes(8, 'big')

    assert blocks5[0] == expected_block5_0, "First block for 64-byte message incorrect"
    assert blocks5[1] == expected_block5_1, "Padding block for 64-byte message incorrect"


    print("All block_parse function test cases passed.")

test_block_parse()

All block_parse function test cases passed.


---

## Problem 4: Hashes

### Instruction
- Write a function `hash(current, block)` that calculates the next hash value given the `current` hash value and the next message `block` according to section *6.2.2 SHA-256 Hash Computation* on page 22 of the Secure Hash Standard.

#### Solution:

In [51]:
def inital_hash_values() -> list[np.uint32]:
    """
    Return the initial hash values for SHA-256 as a list of 32-bit unsigned integers.

    Returns
    -------
        list[numpy.uint32]
            A list of the initial hash values for SHA-256.
    """

    # SHA-256 initial hash values
    # See: FIPS 180-4, Section 5.3.3, p.16
    # https://nvlpubs.nist.gov/nistpubs/fips/nist.fips.180-4.pdf
    H = [
        np.uint32(0x6a09e667),  # √2
        np.uint32(0xbb67ae85),  # √3
        np.uint32(0x3c6ef372),  # √5
        np.uint32(0xa54ff53a),  # √7
        np.uint32(0x510e527f),  # √11
        np.uint32(0x9b05688c),  # √13
        np.uint32(0x1f83d9ab),  # √17
        np.uint32(0x5be0cd19)   # √19
    ]
    
    return H

In [52]:
def message_schedule(block: bytes) -> list[np.uint32]:
    """
    Create the message schedule for a 512-bit block in SHA-256.

    Parameters
    ----------
        block : bytes
            A 64-byte (512-bit) message block.

    Returns
    -------
        list[numpy.uint32]
            A list of 64 32-bit unsigned integers representing the message schedule.

    Raises
    ------
        TypeError
            If the argument is not a byte string.
        ValueError
            If the argument is not 64 bytes long.
    """

    # Type check as before
    if not isinstance(block, bytes):
        raise TypeError("Argument must be a byte string.")
    
    # Ensure block is exactly 64 bytes (512 bits)
    if len(block) != 64:
        raise ValueError("Block must be exactly 64 bytes (512 bits) long.")

    # Initialize the message schedule array with zeros
    W = [np.uint32(0)] * 64

    # First 16 words (W₀ to W₁₅) come directly from the block
    # Split the 512-bit block into 16 chunks of 32 bits (4 bytes each)
    # Use big-endian byte order as specified in FIPS 180-4
    for i in range(16):
        # Extract 4 bytes and convert to a 32-bit unsigned integer
        W[i] = np.uint32(int.from_bytes(block[i*4:(i+1)*4], byteorder='big'))

    # Remaining 48 words (W₁₆ to W₆₃) are calculated using the formula:
    # Wₜ = σ₁(Wₜ₋₂) + Wₜ₋₇ + σ₀(Wₜ₋₁₅) + Wₜ₋₁₆
    # See: FIPS 180-4, Section 6.2.2, step 1
    for i in range(16, 64):
        # Calculate W[i] using the small sigma functions from Problem 1
        W[i] = np.uint32(
            smallSigma1(W[i - 2]) +
            W[i - 7] +
            smallSigma0(W[i - 15]) +
            W[i - 16]
        )
        
    return W

In [53]:
def compress(H: list[np.uint32], W: list[np.uint32]) -> list[np.uint32]:
    """
    SHA-256 compression function - performs 64 rounds of operations.
    
    Parameters
    ----------
    H : list[np.uint32]
        Current hash values (8 32-bit words).
    W : list[np.uint32]
        Message schedule (64 32-bit words from message_schedule()).
    
    Returns
    -------
    list[np.uint32]
        Updated hash values after compression (8 32-bit words).
    
    Raises
    ------
    TypeError
        If H or W are not lists.
    ValueError
        If H doesn't have 8 values or W doesn't have 64 values.
    """
    # Type checks
    if not isinstance(H, list) or not isinstance(W, list):
        raise TypeError("H and W must be lists.")
    
    # Length checks
    if len(H) != 8:
        raise ValueError("H must contain exactly 8 hash values.")
    if len(W) != 64:
        raise ValueError("W must contain exactly 64 words.")
    
    # K constants from Problem 2 (first 32 bits of fractional parts of cube roots)
    # See: FIPS 180-4, Section 4.2.2, page 11
    K = [
        np.uint32(0x428a2f98), np.uint32(0x71374491), np.uint32(0xb5c0fbcf), np.uint32(0xe9b5dba5),
        np.uint32(0x3956c25b), np.uint32(0x59f111f1), np.uint32(0x923f82a4), np.uint32(0xab1c5ed5),
        np.uint32(0xd807aa98), np.uint32(0x12835b01), np.uint32(0x243185be), np.uint32(0x550c7dc3),
        np.uint32(0x72be5d74), np.uint32(0x80deb1fe), np.uint32(0x9bdc06a7), np.uint32(0xc19bf174),
        np.uint32(0xe49b69c1), np.uint32(0xefbe4786), np.uint32(0x0fc19dc6), np.uint32(0x240ca1cc),
        np.uint32(0x2de92c6f), np.uint32(0x4a7484aa), np.uint32(0x5cb0a9dc), np.uint32(0x76f988da),
        np.uint32(0x983e5152), np.uint32(0xa831c66d), np.uint32(0xb00327c8), np.uint32(0xbf597fc7),
        np.uint32(0xc6e00bf3), np.uint32(0xd5a79147), np.uint32(0x06ca6351), np.uint32(0x14292967),
        np.uint32(0x27b70a85), np.uint32(0x2e1b2138), np.uint32(0x4d2c6dfc), np.uint32(0x53380d13),
        np.uint32(0x650a7354), np.uint32(0x766a0abb), np.uint32(0x81c2c92e), np.uint32(0x92722c85),
        np.uint32(0xa2bfe8a1), np.uint32(0xa81a664b), np.uint32(0xc24b8b70), np.uint32(0xc76c51a3),
        np.uint32(0xd192e819), np.uint32(0xd6990624), np.uint32(0xf40e3585), np.uint32(0x106aa070),
        np.uint32(0x19a4c116), np.uint32(0x1e376c08), np.uint32(0x2748774c), np.uint32(0x34b0bcb5),
        np.uint32(0x391c0cb3), np.uint32(0x4ed8aa4a), np.uint32(0x5b9cca4f), np.uint32(0x682e6ff3),
        np.uint32(0x748f82ee), np.uint32(0x78a5636f), np.uint32(0x84c87814), np.uint32(0x8cc70208),
        np.uint32(0x90befffa), np.uint32(0xa4506ceb), np.uint32(0xbef9a3f7), np.uint32(0xc67178f2)
    ]
    
    # Initialize working variables with current hash values
    # See: FIPS 180-4, Section 6.2.2, step 2
    a, b, c, d, e, f, g, h = H
    
    # Perform 64 rounds of compression
    # See: FIPS 180-4, Section 6.2.2, step 3
    for t in range(64):
        # Calculate T1 using h, big sigma 1, Ch function, K constant, and message word
        # T₁ = h + Σ₁(e) + Ch(e,f,g) + Kₜ + Wₜ
        T1 = np.uint32(h + bigSigma1(e) + Ch(e, f, g) + K[t] + W[t])
        
        # Calculate T2 using big sigma 0 and Maj function
        # T₂ = Σ₀(a) + Maj(a,b,c)
        T2 = np.uint32(bigSigma0(a) + Maj(a, b, c))
        
        # Update working variables (rotate down the chain)
        # See: FIPS 180-4, Section 6.2.2, step 3
        h = g
        g = f
        f = e
        e = np.uint32(d + T1)
        d = c
        c = b
        b = a
        a = np.uint32(T1 + T2)
    
    # Compute intermediate hash values by adding compressed values to original
    # See: FIPS 180-4, Section 6.2.2, step 4
    H_new = [
        np.uint32(H[0] + a),
        np.uint32(H[1] + b),
        np.uint32(H[2] + c),
        np.uint32(H[3] + d),
        np.uint32(H[4] + e),
        np.uint32(H[5] + f),
        np.uint32(H[6] + g),
        np.uint32(H[7] + h)
    ]
    
    return H_new

In [54]:
def hash(current: list[np.uint32], block: bytes) -> list[np.uint32]:
    """
    Perform one iteration of the SHA-256 hash function on a given block.

    Parameters
    ----------
        current : list[numpy.uint32]
            Current hash values (8 32-bit words).
        block : bytes
            A 64-byte (512-bit) message block.

    Returns
    -------
        list[numpy.uint32]
            Updated hash values after processing the block.

    Raises
    ------
        TypeError
            If current is not a list or block is not a byte string.
        ValueError
            If current doesn't have 8 values or block doesn't have 64 bytes.
    """

    # Type checks as before
    if not isinstance(current, list):
        raise TypeError("Current hash values must be provided as a list.")
    if not isinstance(block, bytes):
        raise TypeError("Block must be a byte string.")

    # Length checks as before
    if len(current) != 8:
        raise ValueError("Current hash values list must contain exactly 8 values.")
    if len(block) != 64:
        raise ValueError("Block must be exactly 64 bytes (512 bits) long.")

    # Create the message schedule from the block
    W = message_schedule(block)

    # Perform the compression function
    new_hash = compress(current, W)

    # Perform the compression function and return updated hash values
    return new_hash

##### Test Hash Function

In [55]:
def test_hash_components():
    # Test initial hash values
    initial_vals = inital_hash_values()
    assert len(initial_vals) == 8
    assert initial_vals[0] == np.uint32(0x6a09e667)
    assert initial_vals[1] == np.uint32(0xbb67ae85)
    
    # Test message schedule with empty block
    empty_block = b'\x00' * 64
    W = message_schedule(empty_block)
    assert len(W) == 64
    assert W[0] == np.uint32(0)  # First word should be 0
    
    # Test compression function
    H = inital_hash_values()
    W_test = [np.uint32(0)] * 64
    
    # Suppress overflow warnings for expected 32-bit wraparound behavior
    with np.errstate(over='ignore'):
        H_new = compress(H, W_test)
    
    assert len(H_new) == 8
    assert H_new != H  # Should be different after compression
    
    # Test complete hash function
    test_block = b'a' * 64  # 64 bytes of 'a'
    
    with np.errstate(over='ignore'):
        H_result = hash(H, test_block)
    
    assert len(H_result) == 8
    assert all(isinstance(val, np.uint32) for val in H_result)

def test_hash_type_errors():
    # Test message_schedule type errors
    try:
        message_schedule("not bytes")
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for non-bytes input")
    
    try:
        message_schedule(b'short')  # Wrong length
    except ValueError:
        pass
    else:
        raise AssertionError("Expected ValueError for wrong block length")
    
    # Test hash function type errors
    try:
        hash("not list", b'\x00' * 64)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for non-list current")

test_hash_components()
test_hash_type_errors()
print("All test cases pass")

All test cases pass


---

## Problem 5: Passwords

### Problem 5: Password Analysis

The following SHA-256 hashes represent three common passwords that were hashed using a single pass of SHA-256 with UTF-8 encoding:

1. `5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8`
2. `873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34`
3. `b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342`

#### How SHA256 Password Cracking Works
- **Hashes are one-way functions:** SHA256 takes input (like a password) and produces a fixed 256-bit hash. You cannot reverse it directly.

- **Cracking = guessing inputs:** Attackers generate candidate passwords, hash them with SHA256, and compare results to the target hash.

- **GPU acceleration:** Modern GPUs (like NVIDIA RTX cards) can compute billions of hashes per second, making brute-force feasible.
    - Example speeds given:
        - MD5: ~50 billion/sec
        - SHA1: ~15.9 billion/sec
        - SHA256: ~7.1 billion/sec

- **Tools used:** Popular frameworks include Hashcat and John the Ripper, which exploit GPU parallelism.

#### Methods Mentioned
- **Brute Force:** Trying every possible combination until a match is found. Effective for short/simple passwords.

- **Dictionary Attacks:** Using lists of common passwords or leaked credentials to test likely candidates.

- **Hybrid Attacks:** Combining dictionary words with variations (adding numbers, symbols).

- **Rainbow Tables:** Precomputed hash databases (less effective against salted hashes).

- **Network vs Offline Attacks:**
    - **Network-based:** Guessing passwords via login forms (slow, rate-limited).

    - **Offline:** Working directly against a stolen hash (fast, devastating)

#### Why SHA256 Is Harder to Crack
- **Slower than MD5/SHA1:** At ~7.1 billion/sec, SHA256 is computationally heavier, meaning brute force takes longer.
- **Salting:** If a salt is added, precomputed attacks (rainbow tables) become useless, forcing unique brute-force attempts.
- **Password length/complexity:** Strong, long, random passwords are effectively uncrackable even with massive GPU farms.

#### Key Takeaways
- **No “decryption” exists:** Cracking SHA256 is about testing guesses, not reversing the algorithm.
- **GPU farms/cloud instances:** Attackers often scale up with multiple GPUs or cloud services (e.g., AWS EC2 GPU instances).
- **Security lesson:** Use long, unique, salted passwords to make brute-force impractical.


See: [SHA256 Hash Cracking](https://passwordrecovery.io/sha256/)

#### Verification
The following code verifies these passwords using our SHA-256 implementation from Problems 1-4:


In [56]:
def sha256_complete(message: str) -> str:
    """
    Complete SHA-256 hash function using our implementations.
    
    Combines block_parse() from Problem 3, initial_hash() and hash()
    from Problem 4 to produce the final SHA-256 hash.
    
    Parameters
    ----------
    message : str
        The message to hash (will be UTF-8 encoded).
    
    Returns
    -------
    str
        The SHA-256 hash as a 64-character hexadecimal string.
    """
    # Encode the message as UTF-8 bytes
    msg_bytes = message.encode('utf-8')
    
    # Get initial hash values
    H = inital_hash_values()
    
    # Process each 512-bit block with overflow warning suppression
    with np.errstate(over='ignore'):
        for block in block_parse(msg_bytes):
            H = hash(H, block)
    
    # Convert final hash values to hexadecimal string
    # Each of the 8 hash values is formatted as 8 hex characters
    hex_hash = ''.join(format(h, '08x') for h in H)
    
    return hex_hash


# The cracked passwords
passwords = ["password", "cheese", "P@ssw0rd"]

# The original hashes from the problem
expected_hashes = [
    "5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8",
    "873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34",
    "b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342"
]

# Verify each password
print("Verifying cracked passwords using our SHA-256 implementation:\n")
# Loop over each password and its expected hash using enumerate for indexing
for i, (pwd, expected) in enumerate(zip(passwords, expected_hashes), 1):
    # Hash the password using our implementation
    computed = sha256_complete(pwd)
    
    # Check if it matches
    match = computed == expected
    status = "✓ MATCH" if match else "✗ NO MATCH"
    
    print(f"Password {i}: '{pwd}'")
    print(f"  Expected:  {expected}")
    print(f"  Computed:  {computed}")
    print(f"  {status}\n")

print("All passwords verified successfully using our SHA-256 implementation!")

Verifying cracked passwords using our SHA-256 implementation:

Password 1: 'password'
  Expected:  5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8
  Computed:  5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8
  ✓ MATCH

Password 2: 'cheese'
  Expected:  873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34
  Computed:  873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34
  ✓ MATCH

Password 3: 'P@ssw0rd'
  Expected:  b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342
  Computed:  b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342
  ✓ MATCH

All passwords verified successfully using our SHA-256 implementation!


#### Why This Attack Succeeded

The attack succeeded because of several fundamental weaknesses:

- **Offline Hash Recovery:** Once attackers obtain a password hash (from a database leak, intercepted traffic, or a protected file), they can attack it offline. This bypasses rate limits or lockouts that normally protect online login attempts.

- **Raw Computing Power:** Modern GPUs can compute billions of hashes per second. For example, an Nvidia 2080 Ti can crunch about 7.1 billion SHA-256 hashes per second. This makes brute-force or dictionary attacks feasible against weak or common passwords.

- **No Salting:** If the hash wasn’t salted (adding random data before hashing), attackers can precompute or reuse rainbow tables. This dramatically reduces the time needed to crack passwords.

- **Weak Passwords:** Even strong algorithms fail if users choose simple, short, or common passwords. Attackers often succeed because human-chosen passwords are predictable.

- **Hash Reuse Across Systems:** If the same password hash is used in multiple places (e.g., Word documents, ZIP files, or databases), attackers only need to crack it once to gain access across systems

#### Attack Process
1. **Hash Acquisition:** The attacker extracts the SHA-256 hash from a file, database, or intercepted traffic.

2. **Brute Force / Dictionary Attack:** Using GPU clusters or cloud instances, they generate candidate passwords and hash them until one matches.

3. **Success Factors:**

    - High-speed computation (billions of attempts per second).

    - Lack of salting or key-stretching (like PBKDF2, bcrypt, or Argon2).

    - Weak or reused passwords.

#### Why Defenses Failed
- **SHA-256 Alone Isn’t Enough:** It’s designed for speed and integrity, not password security. Fast hashing is great for verifying file integrity but terrible for protecting passwords.

- **No Key-Stretching:** Secure systems use slow, memory-hard algorithms (bcrypt, scrypt, Argon2) to make brute-force attacks impractical. Without these, attackers exploit speed.

- **Human Behavior:** Users often choose passwords that are easy to guess, making brute-force attacks succeed faster.

#### Key Takeaway
The attack succeeded not because SHA-256 is “broken,” but because **passwords were hashed without proper defenses.** Using unsalted, fast hashes leaves passwords vulnerable to brute-force attacks. Stronger practices—like salting, key-stretching, and enforcing complex passwords—are essential to prevent it.


You could even test out the hash passwords using bash: [Test Password using Bash](testPassword.sh)

---

## Conclusion
This notebook has successfully implemented the core components of the SHA-256 cryptographic hash function as specified in FIPS 180-4. Through five interconnected problems, the implementation demonstrates how mathematical principles, bitwise operations, and iterative processing combine to create a secure one-way hash function.

### Summary of Implementation
**Problem 1** established the foundation with seven logical functions (Parity, Ch, Maj, and four Sigma variants) using bitwise operations. These functions, though individually simple, create complex non-linear mixing when combined in the compression function.

**Problem 2** generated the 64 round constants (K values) by extracting fractional parts from cube roots of prime numbers. This mathematical derivation follows the "nothing up my sleeve" principle, providing cryptographic assurance that constants contain no hidden weaknesses.

**Problem 3** implemented the padding scheme that ensures messages of any length are processed as 512-bit blocks. The padding includes the original message length, preventing length-extension attacks and ensuring proper block boundaries.

**Problem 4** brought everything together in the compression function. Through 64 rounds of operations, each message block is mixed with the current hash state using the functions from Problem 1 and constants from Problem 2. The message schedule expands each 512-bit block into 64 words, and the compression function processes these through iterative operations.

**Problem 5** demonstrated a critical security insight: while SHA-256 is excellent for data integrity, it is fundamentally inappropriate for password storage without additional protections. The three passwords were cracked instantly using rainbow tables, demonstrating why modern systems require salted, iterated key derivation functions rather than fast cryptographic hashes.

### Key Insights
- **Mathematical Rigor Meets Practical Security:** SHA-256’s design—bitwise functions, prime-derived constants, and iterative compression—illustrates how simple operations can combine into a robust cryptographic primitive.

- **Strength in Integrity, Weakness in Password Storage:** While SHA-256 is highly secure for verifying data integrity, its speed makes it unsuitable for password protection. Attackers exploit this speed with brute-force and rainbow table attacks.

- **Importance of Salting and Key-Stretching:** Password security requires algorithms like bcrypt, scrypt, or Argon2, which deliberately slow down hashing and incorporate unique salts to prevent precomputed attacks.

- **Human Factor Remains Critical:** Even the strongest algorithms fail if users choose weak or common passwords. Security is a balance of cryptographic design and user behavior.

- **Lesson for System Designers:** Cryptographic primitives must be applied in the right context. SHA-256 is excellent for checksums and digital signatures, but password storage demands specialized, memory-hard functions.

### Reflection
Implementing SHA-256 from the FIPS 180-4 specification underscored the unforgiving precision of cryptographic work. Even the smallest deviation in bitwise operations, byte ordering, or iteration sequence produces results that fail entirely, highlighting why cryptographic code must be validated against authoritative test vectors. This exercise reinforced the discipline required in secure coding: correctness is binary—either exact or broken.

Equally important was the password-cracking demonstration. It revealed the gap between theoretical understanding and practical application. Knowing that SHA-256 is a strong cryptographic hash is not enough; security depends on context. When used for password storage without salting or key-stretching, SHA-256’s speed becomes its weakness, enabling attackers to exploit brute-force and rainbow table attacks. This distinction between **cryptographic strength** and **security suitability** is a critical lesson for system designers.

---

**Note:** All implementations in this notebook follow PEP 257 docstring conventions, include type hints for clarity, and provide comprehensive inline comments. Code is structured for readability and educational value, with each function thoroughly documented and tested against the Secure Hash Standard specifications.




---

## End of the Assessment